# 自定义层
我们可以用创造性的方式组合不同的层，从而设计出适用于各种任务的架构。例如，研究人员发明了专门用于处理图像、文本、序列数据和执行童泰规划的层。

## 不带参数的层
首先构建一个，没有任何参数的自定义层。

In [1]:
import torch
import torch.nn.functional as F
from torch import nn

# 从输入减去输入均值的层
class CenteredLayer(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, X):
        return X - X.mean()

In [2]:
layer = CenteredLayer()
layer(torch.FloatTensor([1, 2, 3, 4, 5]))

tensor([-2., -1.,  0.,  1.,  2.])

现在将层作为组件合并到更复杂的模型中。

In [3]:
net = nn.Sequential(nn.Linear(8, 128), CenteredLayer())

作为额外的安全检查，我们可以在向网络发送随机数据后，检查均值是否为零。由于我们处理的是浮点数，由于储存精度的原因，我们仍然可能会看到一个非常小的非零数。

In [4]:
Y = net(torch.rand(4, 8))
Y.mean()

tensor(-3.7253e-09, grad_fn=<MeanBackward0>)

## 带参数的层
实现自定义版本的全连接层。该层需要两个参数，一个用于权重，一个用于表示偏置项。在此实现中我们使用修正线性单元作为激活函数。该层需要输入参数：`in_units`和`units`, 分别表示输入数和输出数。

In [5]:
class MyLinear(nn.Module):
    def __init__(self, in_units, units):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(in_units, units))
        self.bias = nn.Parameter(torch.randn(units,))
    def forward(self, X):
        linear = torch.matmul(X, self.weight.data) + self.bias.data
        return F.relu(linear)

接下来实例化`MyLinear`类并访问其模型参数。

In [8]:
linear = MyLinear(5, 3)
linear.weight

Parameter containing:
tensor([[ 0.6512, -1.6587,  0.3279],
        [ 0.2026, -1.2441,  1.6832],
        [-1.5194,  0.6381,  0.5577],
        [-2.0067,  1.2290, -0.5103],
        [-0.9141,  0.7939, -0.6044]], requires_grad=True)

可以使用`自定义层直接执行前向传播计算`

In [9]:
linear(torch.rand(2, 5))

tensor([[0.0000, 0.0000, 0.6554],
        [0.0000, 0.0000, 0.9850]])

还可以使用自定义层构建模型，就像是使用内置的全连接层一样使用的自定义层。

In [11]:
net = nn.Sequential(MyLinear(64, 8), MyLinear(8, 1))
net(torch.rand(2, 64))

tensor([[9.6272],
        [6.4779]])